In [ ]:
import json
from privacypacking.utils.utils import load_logs, global_metrics
import pandas as pd
from experiments.ray.analysis import load_ray_experiment, load_latest_ray_experiment, load_latest_scheduling_results, load_latest_scheduling_results, load_latest_ray_experiment
import plotly.express as px
from privacypacking.budget.curves import  LaplaceCurve, GaussianCurve, SubsampledGaussianCurve
from privacypacking.budget import Budget, Task, Block
from privacypacking.schedulers.metrics import OverflowRelevance, FlatRelevance
from privacypacking.budget.block_selection import RandomBlocks
from privacypacking.utils.plot import plot_budgets
import yaml
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from experiments.ray.analysis import get_percentiles

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-11_11-22-47"))
rdf["scheduler_metric"] = rdf.apply(lambda row: row.scheduler_metric if row.scheduler == "basic_scheduler" else "Simplex", axis=1)
px.line(
    rdf.sort_values("n_tasks"),
    x="n_tasks",
    y="n_allocated_tasks",
    color="scheduler_metric",
    log_x=True,
    width=800,
    height=600,
    # range_y=[0,1_500],
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
def map_metric_to_id(row):
    d = {
        "DominantShares": 0,
        "FlatRelevance": 1,
        "OverflowRelevance":2,
        "SoftKnapsack":3,
        "Simplex":4,
    }
    return d[row]

gnuplot_df = rdf.query("scheduler_metric not in ['Fcfs'] and n_tasks not in [8,9,300,500]")
gnuplot_df["id"] = gnuplot_df.scheduler_metric.apply(map_metric_to_id)
gnuplot_df = gnuplot_df[["n_tasks", "n_allocated_tasks", "id", "scheduler", "scheduler_metric"]].sort_values(["id","n_tasks"]).drop_duplicates()
gnuplot_df.to_csv("/home/pierre/privacypacking-paper/results/motivating_examples_simulation/problem_2.csv", index=False)

In [ ]:

rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-11_11-50-03"))
rdf["scheduler_metric"] = rdf.apply(lambda row: row.scheduler_metric if row.scheduler == "basic_scheduler" else "Simplex", axis=1)
px.line(
    rdf.sort_values("n_initial_blocks"),
    x="n_initial_blocks",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    # range_y=[0,1_500],
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)


In [ ]:
def map_metric_to_id(row):
    d = {
        "DominantShares": 0,
        "FlatRelevance": 1,
        "OverflowRelevance":2,
        "SoftKnapsack":3,
        "Simplex":4,
    }
    return d[row]

gnuplot_df = rdf.query("scheduler_metric not in ['Fcfs'] and n_tasks not in [-1]")
gnuplot_df["id"] = gnuplot_df.scheduler_metric.apply(map_metric_to_id)
gnuplot_df = gnuplot_df[["n_initial_blocks", "n_allocated_tasks", "id", "scheduler", "scheduler_metric"]].sort_values(["id","n_initial_blocks"]).drop_duplicates()
gnuplot_df.to_csv("/home/pierre/privacypacking-paper/results/motivating_examples_simulation/problem_1.csv", index=False)